In [1]:
import json
import os
from collections import defaultdict

In [2]:
symcat_conditions = "/Users/teliov/TUD/Thesis/Medvice/Notebooks/data/definitions/symcat_conditions.json"

In [3]:
output_dir = "/Users/teliov/TUD/Thesis/Medvice/Notebooks/data/definitions/"

In [4]:
with open(symcat_conditions) as fp:
    conditions = json.load(fp)

In [5]:
# get all symptoms
symptom_list = set([])
for condition_name in conditions.keys():
    condition_symptoms = set(conditions.get(condition_name).get("symptoms").keys())
    symptom_list = symptom_list | condition_symptoms

In [6]:
from itertools import combinations

In [7]:
from copy import deepcopy

In [8]:
import numpy as np

In [9]:
TARGET = 5

In [10]:
def rand_generation(mode):
    new_conditions = deepcopy(conditions)
    for condition_name in conditions.keys():
        condition_symptoms = set(conditions.get(condition_name).get("symptoms").keys())
        prob_vals = [itm.get("probability") for itm in conditions.get(condition_name).get("symptoms").values()]
        if mode == 1:
            # assign the mean
            new_symptom_prob = np.mean(prob_vals)
        elif mode == 2:
            # assign the max
            new_symptom_prob = np.max(prob_vals)
        elif mode == 3:
            # assign the min
            new_symptom_prob = np.min(prob_vals)
        else:
            # mean * 0.5
            mode = 4
            new_symptom_prob = np.ceil(0.5 * np.mean(prob_vals))

        new_symptom_prob = int(new_symptom_prob)
        symptoms_set = symptom_list - condition_symptoms
        selected_symptoms = np.random.choice(list(symptoms_set), TARGET, replace=False)
        for symp in selected_symptoms:
            new_conditions.get(condition_name).get("symptoms").update({
                symp: {
                    "slug": symp,
                    "probability": new_symptom_prob
                }
            })

    new_conditions_file = os.path.join(output_dir, "symcat_random_%d.json" % mode)
    with open(new_conditions_file, "w") as fp:
        json.dump(new_conditions, fp, indent=4)

In [11]:
rand_generation(1)
rand_generation(2)
rand_generation(3)
rand_generation(4)